In [1]:
import os

In [2]:
%pwd

'f:\\Personal Project\\Medical-Image-Segmentation\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'f:\\Personal Project\\Medical-Image-Segmentation'

In [5]:
from dataclasses import dataclass

from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    copy_data: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from LiverTumorSegmentation.constants import *

In [8]:
print(CONFIG_FILE_PATH)
print(PARAMS_FILE_PATH)

configs\config.yaml
configs\params.yaml


In [9]:
from LiverTumorSegmentation.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        create_directories([config.copy_data])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            copy_data=config.copy_data,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [22]:
import os
import gdown
import zipfile
from LiverTumorSegmentation import logger
from LiverTumorSegmentation.utils.common import get_size
import shutil

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            url = prefix + file_id
            gdown.download(url, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")


        except Exception as e:
            raise e

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    def copy_files(self):
        copy_data = self.config.copy_data
        unzip_path = self.config.unzip_dir
        os.makedirs(copy_data, exist_ok=True)
        for file in os.listdir(unzip_path):
            shutil.copy(os.path.join(unzip_path, file), copy_data)

    

In [ ]:
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  copy_data: src/LiverTumorSegmentation/data/artifacts/data_ingestion
  source_url: https://drive.google.com/file/d/1pTnlnRERzBHNOT2P2gO9Mrql0nDm1H0W/view?usp=sharing
  local_data_file: artifacts/data_ingestion/Medical-Segmentation.zip
  unzip_dir: artifacts/data_ingestion

  

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.copy_files()
except Exception as e:
    raise e
    

[2026-01-20 19:55:18,148: INFO: common: yaml file: configs\config.yaml loaded successfully]
[2026-01-20 19:55:18,150: INFO: common: yaml file: configs\params.yaml loaded successfully]
[2026-01-20 19:55:18,152: INFO: common: created directory at: artifacts]
[2026-01-20 19:55:18,154: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-20 19:55:18,155: INFO: common: created directory at: src/LiverTumorSegmentation/data/artifacts/data_ingestion]
[2026-01-20 19:55:18,156: INFO: 184664710: Downloading data from https://drive.google.com/file/d/1pTnlnRERzBHNOT2P2gO9Mrql0nDm1H0W/view?usp=sharing into file artifacts/data_ingestion/Medical-Segmentation.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1pTnlnRERzBHNOT2P2gO9Mrql0nDm1H0W
To: f:\Personal Project\Medical-Image-Segmentation\artifacts\data_ingestion\Medical-Segmentation.zip
100%|██████████| 18.1M/18.1M [00:11<00:00, 1.58MB/s]

[2026-01-20 19:55:50,219: INFO: 184664710: Downloaded data from https://drive.google.com/file/d/1pTnlnRERzBHNOT2P2gO9Mrql0nDm1H0W/view?usp=sharing into artifacts/data_ingestion/Medical-Segmentation.zip]


NameError: name 'shutil' is not defined